In [1]:
%load_ext autoreload 
%autoreload 2

import os
import random
from collections import deque
import numpy as np
import scipy.linalg as sl
from PIL import Image
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
from IPython import display

import torch
from torch import nn, distributions as dist, autograd
from torch.func import jacfwd
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, Resize, CenterCrop, RandomHorizontalFlip, RandomVerticalFlip, ToTensor, Normalize
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
# torch.set_default_device("cuda")
torch.set_default_dtype(torch.float32)
plt.style.use('seaborn-v0_8')

from mnist import MNISTTrain, MNISTTest

/env_dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_size = 28
batch_size = 512
num_classes = 10

In [3]:
train_ds = MNISTTrain(transform=Compose([
    ToTensor(),
    RandomHorizontalFlip(0.1),
    RandomVerticalFlip(0.1),
    Normalize(127., 128.0)
]))
val_ds = MNISTTest(transform=Compose([
    ToTensor(),
    Normalize(127., 128.0)
]))


In [4]:
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, drop_last=True)

In [5]:
class Sampler:
    
    def __init__(self, buffer_size, img_size):
        self.random_sampler = dist.Uniform(-1., 1.)
        self.buffer = deque(maxlen=buffer_size)
        self.buffer_size = buffer_size
        self.buffer.extend(self.random_sampler.sample((self.buffer_size, 1, img_size, img_size)).unbind())
        self.buffer_size = buffer_size
        
    def add(self, x):
        self.buffer.append(x)
        # self.buffer.append(torch.rand((1, img_size, img_size)))
            
    def clear(self):
        self.buffer.clear()

    def sample(self, size):
        random_sample_probs = np.random.binomial(1, 0.05, (size,))
        num_pos = max(1, sum(random_sample_probs))
        for i in range(num_pos):
            self.buffer[i] = self.random_sampler.sample((1, img_size, img_size)) 
        
        idx = np.random.choice(size, size, replace=False)
        
        return [self.buffer[i] for i in idx]

In [6]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)


class CNNModel(nn.Module):

    def __init__(self, hidden_features=32, out_dim=1, **kwargs):
        super().__init__()
        # We increase the hidden dimension over layers. Here pre-calculated for simplicity.
        c_hid1 = hidden_features//2
        c_hid2 = hidden_features
        c_hid3 = hidden_features*2

        # Series of convolutions and Swish activation functions
        self.cnn_layers = nn.Sequential(
                nn.Conv2d(1, c_hid1, kernel_size=5, stride=2, padding=4), # [16x16] - Larger padding to get 32x32 image
                Swish(),
                nn.Conv2d(c_hid1, c_hid2, kernel_size=3, stride=2, padding=1), #  [8x8]
                Swish(),
                nn.Conv2d(c_hid2, c_hid3, kernel_size=3, stride=2, padding=1), # [4x4]
                Swish(),
                nn.Conv2d(c_hid3, c_hid3, kernel_size=3, stride=2, padding=1), # [2x2]
                Swish(),
                nn.Flatten(),
                nn.Linear(c_hid3*4, c_hid3),
                Swish(),
                nn.Linear(c_hid3, out_dim)
        )

    def forward(self, x):
        x = self.cnn_layers(x).squeeze(dim=-1)
        return x

In [7]:
ebm = torch.load("/mnt/dl/generation/ebm/generation/model/ebm.pth")

In [8]:
ebm.cuda()

CNNModel(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2), padding=(4, 4))
    (1): Swish()
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): Swish()
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): Swish()
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): Swish()
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=256, out_features=64, bias=True)
    (10): Swish()
    (11): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [9]:
class Trainer:
    
    def __init__(self, model, train_loader, 
                 val_loader=None, epochs=0, 
                 savepath=None, K=1, batch_size=1,
                 grad_step_scale=1., eval_epoch=1000000):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.epochs = epochs
        
        self.optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)    
        self.savepath = savepath
        if self.savepath is not None:
            os.makedirs(self.savepath, exist_ok=True)
        self.K = K
        self.batch_size = batch_size
        self.sampler = Sampler(buffer_size=batch_size, 
                                          img_size=img_size)
        self.noise_process = dist.Normal(0., 0.003)
        
        self.lvn_grad_step_scale = grad_step_scale
        self.eval_epoch = eval_epoch
        self.batch_grad_fn = torch.func.vmap(torch.func.grad(self.grad_input_fn))
        
    
    def sample_langevin(self, sample_size=-1, return_img_gen=False, steps=-1):
        if sample_size == -1:
            sample_size = self.batch_size
        if steps == -1:
            steps = self.K
        
        sampler_imgs = self.sampler.sample(sample_size)
        x = torch.stack(sampler_imgs).cuda()
        
        self.model.eval()
        gen_imgs = []
        gen_imgs.append(x.clone().detach().cpu())
        
        with torch.enable_grad():
            for k in range(steps - 1):
                grad = self.batch_grad_fn(x)
                grad = grad.clip(min=-0.03, max=0.03)
                x = x - self.lvn_grad_step_scale * grad + self.noise_process.sample(x.size()).to(torch.device("cuda"))
                x = x.clip(min=-1, max=1.)
                gen_imgs.append(x.clone().detach().cpu())


        if return_img_gen:
            gen_imgs = torch.stack(gen_imgs, 1).cpu()
            return gen_imgs
        
        gen_imgs.clear()
        self.sampler.clear()
        for lx in x.unbind():
            self.sampler.add(lx.detach().cpu())
        
        return x.clone().detach()
    
    
    def grad_input_fn(self, x):
        y = self.model(x.unsqueeze(0))
        return y.sum()
        
    def fit(self, epoch, loader):
        losses = []
        for step, (img, label) in enumerate(loader):
            img = img.cuda()
            xgen = self.sample_langevin()
            self.model.train()
            Edata = self.model(img)
            Etheta = self.model(xgen)
            loss = Edata.mean() - Etheta.mean() +  (Edata.square() + Etheta.square()).mean()
            
            if torch.isnan(loss).item():
                continue
            
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 10.)
            self.optimizer.step()
            losses.append(loss.item())

            if step % 15 == 0:
                print(f"Epoch: {epoch}, step: {step}, loss: {np.mean(losses)}")
           
            if step == 100:
                break
        
        return losses
    
    def train(self):
        self.train_losses = []
        for epoch in range(1, self.epochs + 1):
            losses = self.fit(epoch, train_loader)
            self.train_losses.extend(losses)
            if epoch % self.eval_epoch == 0:
                self.evaluate(epoch)
        
        self.generate_imgs(16)
        return 
    
    def evaluate(self, epoch=None):
        if self.val_loader is None:
            return
        if epoch is None:
            epoch = 0
        self.model.eval()
        re, fe = 0.0, 0.
        with torch.no_grad():
            k = 0
            for (img, label) in self.val_loader:
                real_img = img.cuda()
                fake_img = torch.rand(real_img.size(0), 1, img_size, img_size).cuda()
                
                real_energy = self.model(real_img)
                fake_energy = self.model(fake_img)
                
                re += real_energy.mean().item()
                fe += fake_energy.mean().item()
                # if k == 5:
                #     break
                # k += 1
        print(f"Evaluation epoch {epoch} loss: {re - fe}, real_energy {re}, fake_energy {fe}")
        self.model.train()
    
    def generate_imgs(self, nsample):
        savepath = os.path.join(self.savepath, f"samples")
        os.makedirs(savepath, exist_ok=True)
        with torch.no_grad():
            xgen = self.sample_langevin(sample_size=nsample, return_img_gen=True, steps=256)
            imgs = xgen.permute((0, 1, 3, 4, 2))
            imgs = imgs.squeeze(-1)
            
            imgs *= 128.
            imgs += 127.
            
            imgs = imgs.unbind()
            for n in range(nsample):
                # img_path = os.path.join(savepath, f"{n:03d}")
                # os.makedirs(img_path, exist_ok=True)
                fname = os.path.join(savepath, f"img{n}.jpeg")
                self.save_img(imgs[n], fname)
    
    def save_img(self, imgs, fname, timesteps=16):
        length = imgs.shape[0]
        npic = length // timesteps 
        
        nrow = int(np.sqrt(npic))
        ncol = int(np.sqrt(npic))
        fig, axes = plt.subplots(nrows=nrow, ncols=ncol)
        for i in range(nrow):
            for j in range(ncol):
                axes[i, j].imshow(imgs[(i * nrow + j) *npic])
                # axes[i, j].set_xaxis_off()
        fig.savefig(fname)
        plt.close()

In [10]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

mnist_trainer = Trainer(ebm, 
                        train_loader=train_loader, 
                        val_loader=val_loader,
                        epochs=50, 
                        savepath="/mnt/dl/generation/ebm/generation",
                        batch_size=batch_size,
                        grad_step_scale=10.,
                        K=20,
                        eval_epoch=10,
                        )

In [11]:
mnist_trainer.generate_imgs(16)